First, import necessary modules and start the session. A CLI interface will be shown to setup the sound devices.

In [1]:
from soundgraph.graph import SoundGraph, SoundNode
from soundgraph.utils import start_session
from soundgraph.nodes import AudioReader, Mixer, Sampler, Oscillator, Scaler, MidiInput, MidiMapper
import librosa
import numpy as np

session = start_session()

Starting new session
Use default output devices? [(Y)/n)]
Use default input devices? [(Y)/n)]


Let's make a drum machine. For that, we create a dictionary with the paths to each audio, and another dictionary with binary lists indicating in which division to trigger each sound.

In [2]:
samples = dict(
    kick = '../samples/drumkits/Roland TR-808/Bassdrum-01.mp3',
    hh_closed = '../samples/drumkits/Roland TR-808/Hat Closed.mp3',
    hh_open = '../samples/drumkits/Roland TR-808/Hat Open.mp3',
    snare = '../samples/drumkits/Roland TR-808/Snaredrum.mp3',
    crash = '../samples/drumkits/Roland TR-808/Crash-01.mp3'
)

beat = dict(
    kick =      [1,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0],
    hh_closed = [1,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0],
    hh_open =   [0,0,1,0,0,0,1,0,0,0,1,0,0,0,1,0],
    crash =     [1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
    snare =     [0,0,1,0,0,0,1,0,0,0,1,0,0,0,1,0]
)

Then we create a sampler node, name it 'sampler' and assign each of the samples of the dictionary via update_sample(name, path), and the beat via update('beat', beat).
Each node has the update(parameter, value) method, which allows to change the value of a parameter.

In [3]:
sampler = Sampler(name='sampler')
for k,v in samples.items():
    sampler.update_sample(k,v)
    
sampler.update('beat',beat)

/home/lpepino/anaconda3/lib/python3.7/site-packages/librosa/core/audio.py:162: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")
/home/lpepino/anaconda3/lib/python3.7/site-packages/librosa/core/audio.py:162: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")
/home/lpepino/anaconda3/lib/python3.7/site-packages/librosa/core/audio.py:162: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")
/home/lpepino/anaconda3/lib/python3.7/site-packages/librosa/core/audio.py:162: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")
/home/lpepino/anaconda3/lib/python3.7/site-packages/librosa/core/audio.py:162: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audior

Then, we have to instantiate a **SoundGraph**, which is a collection of **SoundNodes**. We add the sampler via **add()**, and set the output of the SoundGraph via **set_output(node_dir)**. Each node and its parameters and outputs have a unique identifier. In this case, the output of the sampler is 'sampler.out', and in general, the pattern is 'node_name.parameter/output'. The output, unless there are multiple outputs, is called 'out'. This output is the node which will feed the output device and we'll listen to.

Finally, we assign the graph to the session via **set_graph(SoundGraph)**

In [4]:
sound_graph = SoundGraph()
sound_graph.add(sampler)
sound_graph.set_output('sampler.out')
session.set_graph(sound_graph)

Now, you should be listening to a drum beat. Let's change parameters on the fly, we will change the bpm, divisions_per_bar, bars, and the beat, using the **update(parameter,value)** method. You can change whatever you want in this cell once and again, and the changes will be reflected in the sounds you are listening almost instantaneously.

In [5]:
sampler.update('bpm',70)
sampler.update('divisions_per_bar',4)
sampler.update('bars',4)

beat = dict(
    kick =      [1,0,0,1,0,1,1,1,0,1,0,0,1,1,1,1],
    hh_closed = [1,0,1,0,0,0,1,0,1,1,0,0,1,0,1,0],
    hh_open =   [0,0,0,0,1,1,0,1,0,0,1,0,1,0,0,0],
    crash =     [1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
    snare =     [1,0,0,0,1,0,0,0,0,0,1,1,1,1,0,0]
)

sampler.update('beat',beat)

Let's add an oscillator.

In [6]:
osc1 = Oscillator(name='osc1')
osc1.update('frequency',220)
sound_graph.add(osc1)

We need to connect it somehow to the output, without removing the drumbeat, for that we need a mixer which takes the sampler and the oscillator outputs as input. We make these connections using the method **connect(source, destination)**

In [7]:
mixer = Mixer(name='mixer')
sound_graph.add(mixer)
sound_graph.connect('osc1.out','mixer.in_1')
sound_graph.connect('sampler.out','mixer.in_2')
mixer.update('volume_1',0.3)
mixer.update('volume_2',0.7)
sound_graph.set_output('mixer.out')

We can change the frequency, amplitude, offset and oscillator shape

In [8]:
osc1.update('osc_type','square')
osc1.update('frequency',110)
osc1.update('amplitude',0.4)
osc1.update('offset',0)

Let's make it more interesting by adding a LFO, which will modulate the oscillator frequency. This happens in the line ``sound_graph.connect('lfo1.out','osc1.frequency')``

In [9]:
lfo1 = Oscillator(name='lfo1')
sound_graph.add(lfo1)
sound_graph.connect('lfo1.out','osc1.frequency')

lfo1.update('frequency',10)
lfo1.update('amplitude',20)
lfo1.update('offset',110)
lfo1.update('osc_type','saw')
osc1.update('osc_type','square')
mixer.update('volume_1',0.08)
mixer.update('volume_2',0.9)

Now, the offset of the LFO controls the central frequency of the oscillator, the amplitude its variation around that center, and the frequency, the rate at which the frequency oscillates.

In [13]:
lfo1.update('frequency',0.857)
lfo1.update('amplitude',220)
lfo1.update('offset',440)

Let's add midi controls! First, let's look at our available MIDI devices:

In [14]:
import mido

mido.get_input_names()

['Midi Through:Midi Through Port-0 14:0',
 'iO|2:iO|2 MIDI 1 16:0',
 'Impact LX61+:Impact LX61+ MIDI 1 20:0',
 'Impact LX61+:Impact LX61+ MIDI 2 20:1',
 'Midi Through:Midi Through Port-0 14:0',
 'iO|2:iO|2 MIDI 1 16:0',
 'Impact LX61+:Impact LX61+ MIDI 1 20:0']

We will use the 'Impact LX61+:Impact LX61+ MIDI 2 20:1' device. Then, we have to recognize the control that we want to use. Run the next cell, and then move the desired control

In [15]:
midi_in = mido.open_input('Impact LX61+:Impact LX61+ MIDI 2 20:1')
midi_in.receive()

Message('control_change', channel=15, control=63, value=92, time=0)

Once the control is changed, a message is printed. We can see the channel and control number we moved.

Now, let's connect the control to the LFO offset. We have 2 nodes: **MidiInput** and the **MidiMapper**. The MidiInput listens to a midi port for messages. We indicate the midi device using the **port** parameter. The MidiMapper looks at those midi messages and filters them. In this case, the freq_mapper, will look for messages of the control 63 in the channel 15. This control can take 128 different values (8 bits) in the 'value' parameter. With mapping, we indicate how to interpret each of those values, in this case, the range 0-128 is turned into a logarithmically spaced range between 20-8000, so by turning our knob we will be able to change the lfo1 offset in a range of 20-8000 Hz.

In [28]:
lx61_controls = MidiInput(name='lx61_controls')
lx61_controls.update('port','Impact LX61+:Impact LX61+ MIDI 2 20:1')
freq_mapping = MidiMapper(name='freq_mapper')
freq_mapping_config = dict(
    channel=15,
    control=63,
    map_parameter='value',
    mapping=np.logspace(1,np.log(8000)/np.log(20),128,base=20)
)
freq_mapping.update('mapping',freq_mapping_config)
sound_graph.add(freq_mapping)
sound_graph.add(lx61_controls)
sound_graph.connect('lx61_controls.out','freq_mapper.midi_in')
sound_graph.connect('freq_mapper.out','lfo1.offset')

Let's add another midi control to change the LFO frequency:

In [27]:
lfo_mapping = MidiMapper(name='lfo_mapper')
lfo_mapping_config = dict(
    channel=15,
    control=62,
    map_parameter='value',
    mapping=np.linspace(0.01,200,128)
)
lfo_mapping.update('mapping',lfo_mapping_config)
sound_graph.add(lfo_mapping)
sound_graph.connect('lx61_controls.out','lfo_mapper.midi_in')
sound_graph.connect('lfo_mapper.out','lfo1.frequency')

Now let's assign some faders to control the mix between the drums and the oscillator:

In [18]:
mix1_mapping = MidiMapper(name='mix1_mapper')
mix1_mapping_config = dict(
    channel=15,
    control=38,
    map_parameter='value',
    mapping=np.linspace(0,1,128),
    last_state=0.08
)
mix1_mapping.update('mapping',mix1_mapping_config)
sound_graph.add(mix1_mapping)
sound_graph.connect('lx61_controls.out','mix1_mapper.midi_in')
sound_graph.connect('mix1_mapper.out','mixer.volume_1')

mix2_mapping = MidiMapper(name='mix2_mapper')
mix2_mapping_config = dict(
    channel=15,
    control=39,
    map_parameter='value',
    mapping=np.linspace(0,1,128),
    last_state=0.9
)
mix2_mapping.update('mapping',mix2_mapping_config)
sound_graph.add(mix2_mapping)
sound_graph.connect('lx61_controls.out','mix2_mapper.midi_in')
sound_graph.connect('mix2_mapper.out','mixer.volume_2')

Something more experimental! Let's control the LFO parameters with the drumbeat. For example, we can connect the sampler output to the LFO offset and amplitude. However, the drumbeat is a waveform with a range of -1 to 1, which will be an inaudible change in the LFO parameters. To address this problem, we add some **Scaler** in between, which multiplies the sampler output.

In [26]:
scaler_freq = Scaler(name='scale_sampler_freq')
scaler_amp = Scaler(name='scale_sampler_amp')
scaler_freq.update('scale',1000)
scaler_amp.update('scale',100)
sound_graph.add(scaler_freq)
sound_graph.add(scaler_amp)
sound_graph.connect('sampler.out','scale_sampler_freq.input')
sound_graph.connect('sampler.out','scale_sampler_amp.input')
sound_graph.connect('scale_sampler_freq.out','lfo1.offset')
sound_graph.connect('scale_sampler_amp.out','lfo1.amplitude')

In [25]:
scaler_freq.update('scale',3000)
scaler_amp.update('scale',1000)
lfo1.update('frequency',0.05)
lfo1.update('osc_type','sin')